In [1]:
%pip install langchain boto3 langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 30.8 MB/s eta 0:00:0000:0100:01
  Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Uninstalling urllib3-2.2.2:
      Successfully uninstalled urllib3-2.2.2

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain import OpenAI
import json
from typing import List, Dict
import random

from main import Student, Curriculum, TeacherInput
import csv

# Initialize the OpenAI model
llm = OpenAI(temperature=0.0)


def save_combined_data_to_csv(data: Dict, subjects: List[str]):
    with open('combined_student_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)

        # Define the header
        headers = ['Student ID', 'Name', 'Grade']
        for subject in subjects:
            headers.append(f'{subject} Score')
            headers.append(f'{subject} Learning Objectives')
        headers.extend(['Focus Areas', 'Challenges'])

        writer.writerow(headers)

        # Write data for each student
        for student in data['students']:
            row = [student.student_id, student.name, student.grade]

            # Add test scores and curriculum objectives for each subject
            for subject in subjects:
                score = student.test_scores.get(subject, 'N/A')
                curriculum_objectives = next(
                    (c.learning_objectives for c in data['curriculums'] if
                     c.subject == subject and c.grade == student.grade), []
                )
                curriculum_str = "; ".join(curriculum_objectives)

                row.append(score)
                row.append(curriculum_str)

            # Add teacher input
            teacher_input = next(
                (ti for ti in data['teacher_inputs'] if ti.student_id == student.student_id), None
            )
            if teacher_input:
                focus_areas_str = "; ".join(teacher_input.focus_areas)
                challenges_str = "; ".join(teacher_input.challenges)
            else:
                focus_areas_str = ''
                challenges_str = ''

            row.extend([focus_areas_str, challenges_str])

            writer.writerow(row)


def generate_dynamic_data(num_students: int, subjects: List[str], grades: List[int]) -> Dict:
    generated_data = {
        "students": [],
        "curriculums": [],
        "teacher_inputs": []
    }

    for i in range(num_students):
        grade = random.choice(grades)

        # Generate student profile, test scores, and curriculum in one JSON response
        prompt = f"""
        Generate a JSON object with the following keys:
        - "student": A student profile with "name", "interests", and "description" for a student in grade {grade}.
        - "test_scores": A list of dictionaries, each with "subject" and "score" keys, representing test scores for the subjects {', '.join(subjects)}.
        - "curriculum": A dictionary where each key is a subject and the value is a list of learning objectives for that subject in grade {grade}.
        """
        response = llm(prompt)
        data = json.loads(response)

        # Extract student information
        student_name = data['student']['name']
        student_id = f"STU{str(i + 1).zfill(3)}"
        student = Student(student_id, student_name, grade)

        # Add test scores to the student
        for score in data['test_scores']:
            student.add_test_score(score['subject'], score['score'])

        generated_data["students"].append(student)

        # Add curriculum information
        for subject, objectives in data['curriculum'].items():
            generated_data["curriculums"].append(Curriculum(subject, grade, objectives))

        # Generate teacher input
        teacher_input_prompt = f"""
        Generate a JSON object with the following keys:
        - "focus_areas": A list of focus areas for the student named {student_name}.
        - "challenges": A list of challenges the student faces in their studies.
        """
        teacher_input_response = llm(teacher_input_prompt)
        teacher_input_data = json.loads(teacher_input_response)

        teacher_input = TeacherInput(
            student_id,
            teacher_input_data['focus_areas'],
            teacher_input_data['challenges']
        )
        generated_data["teacher_inputs"].append(teacher_input)

    return generated_data


# Generate a dataset with 5 students, 2 subjects, and grades ranging from 5 to 6
subjects = ["Math", "Science"]
grades = [5, 6]
dynamic_data = generate_dynamic_data(20, subjects, grades)

# Save the combined data to a single CSV file
save_combined_data_to_csv(dynamic_data, subjects)


/Users/chris/PycharmProjects/edtech_genai/main.py:8: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(temperature=0.0)
/Users/chris/PycharmProjects/edtech_genai/main.py:63: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  lesson_plan_chain = LLMChain(llm=llm, prompt=lesson_plan_prompt)
/Users/chris/PycharmProjects/edtech_genai/main.py:76: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  lesson_plan = lesson_plan_chain.run(


Generated Lesson Plan:
 
Student: John Doe
Grade: 5

Lesson Plan:

Subject: Math
Objective: To improve understanding and mastery of addition, subtraction, multiplication, and division.

Topics to cover:
1. Addition with regrouping
2. Subtraction with borrowing
3. Multiplication tables
4. Division with remainders

Recommended Activities:
1. Use manipulatives such as blocks or counters to demonstrate addition and subtraction.
2. Practice multiplication tables through games and flashcards.
3. Use real-life examples to understand division with remainders.
4. Create word problems to apply the concepts learned.

Additional Resources:
1. Online math games and activities.
2. Math workbooks with practice problems.
3. Khan Academy videos on addition, subtraction, multiplication, and division.

Subject: Science
Objective: To develop an understanding of biology, chemistry, and physics.

Topics to cover:
1. Parts of a plant and their functions
2. States of matter
3. Forces and motion

Recommended A

/var/folders/qn/q1vklts93wd1lk0svgnszsnh0000gn/T/ipykernel_12278/3163190833.py:75: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(prompt)


In [3]:
import csv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from langchain import OpenAI
from typing import List, Dict

llm = OpenAI(temperature=0.0)


# Student class to store student-related data
class Student:
    def __init__(self, student_id: str, name: str, grade: int):
        self.student_id = student_id
        self.name = name
        self.grade = grade
        self.test_scores = {}  # Dictionary to hold test scores by subject
        self.curriculum = {}  # Dictionary to hold curriculum objectives by subject
        self.focus_areas = []
        self.challenges = []

    def add_test_score(self, subject: str, score: float):
        self.test_scores[subject] = score

    def add_curriculum(self, subject: str, learning_objectives: List[str]):
        self.curriculum[subject] = learning_objectives


# Curriculum class to store curriculum standards
class Curriculum:
    def __init__(self, subject: str, grade: int, learning_objectives: List[str]):
        self.subject = subject
        self.grade = grade
        self.learning_objectives = learning_objectives


# TeacherInput class to store teacher-specific inputs for each student
class TeacherInput:
    def __init__(self, student_id: str, focus_areas: List[str], challenges: List[str]):
        self.student_id = student_id
        self.focus_areas = focus_areas
        self.challenges = challenges


# Define the prompt template for generating lesson plans
lesson_plan_prompt = PromptTemplate(
    input_variables=["student_name", "grade", "test_scores", "curriculum", "focus_areas", "challenges"],
    template="""
    You are an expert educator creating a personalized lesson plan for a student.
    The student's name is {student_name}, they are in grade {grade}.

    The student's test scores are as follows: {test_scores}.
    The curriculum objectives for this grade are: {curriculum}.

    The student needs to focus on the following areas: {focus_areas}.
    They also face the following challenges: {challenges}.

    Based on this information, generate a detailed lesson plan for the student, including specific topics to cover, recommended activities, and any additional resources.
    """
)

# Create the LangChain LLM chain
lesson_plan_chain = LLMChain(llm=llm, prompt=lesson_plan_prompt)


def generate_lesson_plan_for_student(student: Student) -> str:
    # Prepare input data for the model
    student_name = student.name
    grade = student.grade
    test_scores = json.dumps(student.test_scores)
    curriculum_str = json.dumps(student.curriculum)
    focus_areas = student.focus_areas
    challenges = student.challenges

    # Generate the lesson plan
    lesson_plan = lesson_plan_chain.run(
        student_name=student_name,
        grade=grade,
        test_scores=test_scores,
        curriculum=curriculum_str,
        focus_areas=focus_areas,
        challenges=challenges
    )

    return lesson_plan


# Example usage:
# Generate a lesson plan for the first student in the dataset
with open("combined_student_data.csv", 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    dynamic_data = [row for row in reader]

student_data = dynamic_data[0]

student_id = student_data['Student ID']
name = student_data['Name']
grade = int(student_data['Grade'])

# Create the student object
student = Student(student_id, name, grade)

# Add focus areas and challenges
student.focus_areas = student_data['Focus Areas'].split('; ')
student.challenges = student_data['Challenges'].split('; ')

# Add test scores
if 'Math Score' in student_data:
    student.add_test_score('Math', float(student_data['Math Score']))
if 'Science Score' in student_data:
    student.add_test_score('Science', float(student_data['Science Score']))

# Add curriculum (parsed from the CSV fields)
if 'Math Learning Objectives' in student_data:
    student.add_curriculum('Math', student_data['Math Learning Objectives'].split('; '))
if 'Science Learning Objectives' in student_data:
    student.add_curriculum('Science', student_data['Science Learning Objectives'].split('; '))

# Generate the lesson plan
lesson_plan = generate_lesson_plan_for_student(student)

print("Generated Lesson Plan:\n", lesson_plan)


Generated Lesson Plan:
 
Student Name: John Doe
Grade: 6

Lesson Plan:

Subject: Math
Objective: By the end of this lesson, John will be able to solve equations with variables on both sides, graph linear equations, and solve problems involving ratios and proportions.

Topics to Cover:
1. Solving equations with variables on both sides
2. Graphing linear equations
3. Solving problems involving ratios and proportions

Recommended Activities:
1. Review the concept of variables and how they are used in equations.
2. Practice solving equations with variables on both sides using examples and guided practice.
3. Introduce the concept of graphing linear equations and provide examples for John to practice.
4. Use real-life scenarios to demonstrate how ratios and proportions are used in problem-solving.
5. Provide John with practice problems to solve on his own, with guidance and support as needed.

Additional Resources:
1. Online tutorials and videos on solving equations with variables on both s